# Biostat 257 HW2
### Caesar Z. Li
### UID: 704135662

## Q1. (Optional, 30 bonus pts) Derivatives

In [1]:
# load necessary packages; make sure install them first
using BenchmarkTools, CSV, DataFrames, DelimitedFiles, Distributions
using Ipopt, LinearAlgebra, MathProgBase, MixedModels, NLopt
using Random, RCall, Revise

## Q2. (20 pts) Objective and gradient evaluator for a single datum

In [2]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # arrays for holding gradient
    ∇β         :: Vector{T}
    ∇σ²        :: Vector{T}
    ∇Σ         :: Matrix{T}    
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    xty        :: Vector{T}
    zty        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    storage_q2 :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
    storage_qq2 :: Matrix{T}
    storage_qq3 :: Matrix{T}
end

In [88]:

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
    ) where T <: AbstractFloat
    n, p, q    = size(X, 1), size(X, 2), size(Z, 2)    
    ∇β         = Vector{T}(undef, p)
    ∇σ²        = Vector{T}(undef, 1)
    ∇Σ         = Matrix{T}(undef, q, q)    
    yty        = abs2(norm(y))
    xty        = transpose(X) * y
    zty        = transpose(Z) * y    
    storage_p  = Vector{T}(undef, p)
    storage_q  = Vector{T}(undef, q)
    storage_q2 = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    storage_qq2 = similar(ztz)
    storage_qq3 = similar(ztz)
    LmmObs(y, X, Z, ∇β, ∇σ², ∇Σ, 
        yty, xty, zty, storage_p, storage_q, storage_q2, 
        xtx, ztx, ztz, storage_qq, storage_qq2, storage_qq3)
end

"""
    logl!(obs::LmmObs, β, L, σ², needgrad=false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `L`, 
and `σ²`. If `needgrad==true`, then `obs.∇β`, `obs.∇Σ`, and `obs.σ² are filled 
with the corresponding gradient.
"""
function logl!(
        obs      :: LmmObs{T}, 
        β        :: Vector{T}, 
        L        :: Matrix{T}, 
        σ²       :: T,
        needgrad :: Bool = true
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    ####################
    # Evaluate objective
    ####################    
    # form the q-by-q matrix: M = σ² * I + Lt Zt Z L
     copy!(obs.storage_qq, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq) # O(q^3)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    # cholesky on M = σ² * I + Lt Zt Z L
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.storage_q, obs.zty)) # O(pq)
    BLAS.trmv!('L', 'T', 'N', L, obs.storage_q)    # O(q^2)
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, obs.storage_q) # O(q^3)
    # l2 norm of residual vector
    copy!(obs.storage_p, obs.xty)
    rtr  = obs.yty +
        dot(β, BLAS.gemv!('N', T(1), obs.xtx, β, T(-2), obs.storage_p))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (rtr - qf) / σ² 
    logl /= -2
    ###################
    # Evaluate gradient
    ###################    
    if needgrad
        # TODO: fill ∇β, ∇L, ∇σ² by gradients
        # form ∇β first:
        # fill in ∇β with Xt(y - Xβ) = xty - XtXβ
        copy!(obs.∇β, obs.xty)
        BLAS.gemv!('N', T(-1), obs.xtx, β, T(1), obs.∇β)
        # storage_q2 = (Mchol.U) \ ( (Mchol.U') \ (Lt * (Zt * res)) )
        copy!(obs.storage_q2, obs.storage_q)
        BLAS.trsv!('U', 'N', 'N', obs.storage_qq, obs.storage_q2)
        BLAS.trmv!('L', 'N', 'N', L, obs.storage_q2) 
        # Put two pieces together, while makeing storage_q2 .= XtZ * storage_q2
        BLAS.gemv!('T', T(-1), obs.ztx, obs.storage_q2, T(1), obs.∇β)
        obs.∇β ./= σ²
        
        # form ∇σ²:
        # expand rtΩ⁻²r into rtr + ( rt * ZLM⁻¹ Lt ) * ZtZ * ( LM⁻¹LtZt * r ) +
        #  rt * ZLM⁻¹ LtZt * r 
        # Note the third term is already computed as 'qf' in earlier code
        # rtr is computed as well, so I need only compute the second term.
        BLAS.gemv!('N', T(1), obs.ztz, obs.storage_q2, T(0), obs.storage_q)
        obs.∇σ² .= dot(obs.storage_q2, obs.storage_q)
        obs.∇σ² .+= rtr - 2qf
        obs.∇σ² ./= σ²
        # minus tr(Ω⁻¹)
        obs.storage_qq2 .= inv(UpperTriangular(obs.storage_qq))
        copy!(obs.storage_qq3, obs.ztz)
        BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq3)
        BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq3)
        BLAS.trmm!('R', 'U', 'N', 'N', T(1), obs.storage_qq2, obs.storage_qq3)
        obs.∇σ² .+= dot(obs.storage_qq2, obs.storage_qq3)
        obs.∇σ² .-= n 
        obs.∇σ² ./= (2 * σ²)
        
        # form ∇Σ:
        # first part, get (-ZtZ + ZtZ * LM⁻¹Lt * ZtZ) L
        # put first half in obs.storage_qq2
        copy!(obs.storage_qq3, L)
        BLAS.trmm!('R', 'U', 'N', 'N', T(1), obs.storage_qq2, obs.storage_qq3)
        BLAS.gemm!('N', 'T', T(1), obs.storage_qq3, obs.storage_qq3, T(0), obs.∇Σ)
        BLAS.gemm!('N', 'N', T(1), obs.ztz, obs.∇Σ, T(0), obs.storage_qq3)
        @inbounds for j in 1:q
             obs.storage_qq3[j, j] -= 1.0
        end
        BLAS.gemm!('N', 'N', T(1), obs.storage_qq3, obs.ztz, T(0), obs.∇Σ)
        BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.∇Σ)
        
        #BLAS.gemm!('N', 'N', T(1), obs.ztz, L, T(0), obs.storage_qq3)
        #BLAS.gemm!('N', 'N', T(1), obs.storage_qq3, obs.storage_qq2, T(0), obs.∇Σ)
        #BLAS.gemm!('N', 'T', T(1), obs.∇Σ, obs.∇Σ, T(0), obs.storage_qq3)
        #BLAS.gemm!('N', 'N', T(1), obs.storage_qq3, L, T(0), obs.∇Σ)       
        #BLAS.gemm!('N', 'N', T(- 1), obs.ztz, L, T(1), obs.∇Σ)
        
        # second part, use Ω⁻¹r from earlier to get ZΩ⁻¹r 
        copy!(obs.storage_q, obs.zty)
        BLAS.gemv!('N', T(- 1), obs.ztx, β, T(1), obs.storage_q)
        # we have storage_q2 = L(Mchol.U) \ ( (Mchol.U') \ (Lt * (Zt * res)) )
        # Put two pieces together, only this time storage_q2 .= ZtZ * storage_q2
        BLAS.gemv!('N', T(- 1), obs.ztz, obs.storage_q2, T(1), obs.storage_q)
        # outer product this vector, then * L
        BLAS.gemm!('N', 'T', T(1), obs.storage_q, obs.storage_q, T(0), obs.storage_qq)
        BLAS.gemm!('N', 'N', T(1 / σ²), obs.storage_qq, L, T(1), obs.∇Σ)
        obs.∇Σ ./= σ²
        
        #sleep(1e-3) # pretend this step takes 1ms
    end    
    ###################
    # Return
    ###################        
    return logl    
end

logl!

In [11]:
@show (obs.xty .- obs.xtx * β - obs.ztx' * L * inv(obs.storage_qq) * obs.storage_q )/σ²

((obs.xty .- obs.xtx * β) - (obs.ztx)' * L * inv(obs.storage_qq) * obs.storage_q) / σ² = [-234.55783448051343, 23.315710964270618, 209.356779599957, 132.02138431384148, -9.009747216169743]


5-element Array{Float64,1}:
 -234.55783448051343 
   23.315710964270618
  209.356779599957   
  132.02138431384148 
   -9.009747216169743

In [73]:
a = - Z' * (inv(σ² * I + Z * Σ * Z')) * Z * L +
 Z' * (inv(σ² * I + Z * Σ * Z')) * (y - X * β) * (y - X * β)' * (inv(σ² * I + Z * Σ * Z')) * Z * L

3×3 Array{Float64,2}:
  1.83101     1.82142     0.0607169
  1.84331     0.116693    0.0714718
 -0.0332479  -0.0202561  -1.00808  

In [74]:
a * a'

3×3 Array{Float64,2}:
  6.67385   3.592     -0.15898 
  3.592     3.41652   -0.135699
 -0.15898  -0.135699   1.01775 

In [64]:
obs.storage_qq / σ²^2

3×3 Array{Float64,2}:
  2.66011     1.73256    -0.0311919 
  1.73256     1.12844    -0.0203157 
 -0.0311919  -0.0203157   0.00036575

In [53]:
 - Z' * (I - Z * L * obs.storage_qq2 * obs.storage_qq2' * L' * Z') * Z * L / σ²

3×3 Array{Float64,2}:
 -0.999238     0.100362     0.0916239
 -6.59332e-5  -1.00425      0.0916019
 -6.10366e-5  -7.53052e-5  -1.00845  

It is a good idea to test correctness and efficiency of the single datum objective/gradient evaluator here. First generate the same data set as in HW2.

In [8]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### 2.1  Correctness

In [72]:
@show logl = logl!(obs, β, L, σ², true)
@show obs.∇β
@show obs.∇σ²
@show obs.∇Σ;

logl = logl!(obs, β, L, σ², true) = -3247.4568580638256
obs.∇β = [-1.6309843232714532, -77.527515580418, -14.702372116010645, 6.978485518990293, -57.711823176822044]
obs.∇σ² = [-4.820377758258701]
obs.∇Σ = [1.8310092226973647 1.8214186972659292 0.06071688596771451; 1.843309519464353 0.11669335312204465 0.07147176879465071; -0.03324793384975084 -0.020256067839013816 -1.0080835623768392]


You will lose all 20 points if following statement throws `AssertionError`.

In [218]:
@assert abs(logl - (-3247.4568580638247)) < 1e-8
@assert norm(obs.∇β - [-1.63098432327115, -77.52751558041871, -14.70237211601065, 
        6.978485518989568, -57.71182317682199]) < 1e-8
@assert norm(obs.∇Σ - [1.6423791649290531 1.82502407722348 0.06127650043330721; 
        1.82502407722348 0.1107239137055005 0.07213050869971993; 
        0.06127650043330721 0.07213050869971993 -1.0173748515299939]) < 1e-8
@assert abs(obs.∇σ²[1] - (-4.8203777582588145)) < 1e-8

AssertionError: AssertionError: norm(obs.∇Σ - [1.6423791649290531 1.82502407722348 0.06127650043330721; 1.82502407722348 0.1107239137055005 0.07213050869971993; 0.06127650043330721 0.07213050869971993 -1.0173748515299939]) < 1.0e-8

### 2.2  Efficiency
Benchmark for evaluating objective only. This is what we did in HW2.

In [75]:
@benchmark logl!($obs, $β, $L, $σ², false)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.786 μs (0.00% GC)
  median time:      1.806 μs (0.00% GC)
  mean time:        1.811 μs (0.00% GC)
  maximum time:     4.995 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [89]:
bm_objgrad = @benchmark logl!($obs, $β, $L, $σ², true)

BenchmarkTools.Trial: 
  memory estimate:  176 bytes
  allocs estimate:  2
  --------------
  minimum time:     7.287 μs (0.00% GC)
  median time:      7.359 μs (0.00% GC)
  mean time:        7.515 μs (0.00% GC)
  maximum time:     20.135 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     4

My median runt time is 2.1μs. You will get full credit (10 pts) if the median run time is within 10μs.

In [90]:
#  The points you will get are
clamp(10 / (median(bm_objgrad).time / 1e3) * 10, 0, 10)

10.0

## Q3. LmmModel type
We create a `LmmModel` type to hold all data points and model parameters. Log-likelihood/gradient of a LmmModel object is simply the sum of log-likelihood/gradient of individual data points.

In [ ]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat} <: MathProgBase.AbstractNLPEvaluator
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β    :: Vector{T}
    L    :: Matrix{T}
    σ²   :: Vector{T}    
    # arrays for holding gradient
    ∇β   :: Vector{T}
    ∇σ²  :: Vector{T}
    ∇L   :: Matrix{T}
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty  :: Vector{T}
    ztr2 :: Vector{T}
    xtx  :: Matrix{T}
    ztz2 :: Matrix{T}
end

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p    = size(obsvec[1].X, 2)
    q    = size(obsvec[1].Z, 2)
    # parameters
    β    = Vector{T}(undef, p)
    L    = Matrix{T}(undef, q, q)
    σ²   = Vector{T}(undef, 1)    
    # gradients
    ∇β   = similar(β)    
    ∇σ²  = similar(σ²)
    ∇L   = similar(L)
    # intermediate arrays
    xty  = Vector{T}(undef, p)
    ztr2 = Vector{T}(undef, abs2(q))
    xtx  = Matrix{T}(undef, p, p)
    ztz2 = Matrix{T}(undef, abs2(q), abs2(q))
    LmmModel(obsvec, β, L, σ², ∇β, ∇σ², ∇L, xty, ztr2, xtx, ztz2)
end

"""
    logl!(m::LmmModel, needgrad=false)

Evaluate the log-likelihood of an LMM model at parameter values `m.β`, `m.L`, 
and `m.σ²`. If `needgrad==true`, then `m.∇β`, `m.∇Σ`, and `m.σ² are filled 
with the corresponding gradient.
"""
function logl!(m::LmmModel{T}, needgrad::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    if needgrad
        fill!(m.∇β , 0)
        fill!(m.∇L , 0)
        fill!(m.∇σ², 0)        
    end
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        logl += logl!(obs, m.β, m.L, m.σ²[1], needgrad)
        if needgrad
            BLAS.axpy!(T(1), obs.∇β, m.∇β)
            BLAS.axpy!(T(1), obs.∇Σ, m.∇L)
            m.∇σ²[1] += obs.∇σ²[1]
        end
    end
    # obtain gradient wrt L: m.∇L = m.∇L * L
    if needgrad
       # TODO 
    end
    logl
end

## Q4. (20 pts) Test data
Let's generate a fake longitudinal data set to test our algorithm.

In [ ]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue)).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);


For later comparison with other software, we save the data into a text file lmm_data.txt. Do not put this file in Git. It takes 246.6MB storage.

In [ ]:
(isfile("lmm_data.txt") && filesize("lmm_data.txt") == 246638945) || 
open("lmm_data.txt", "w") do io
    p = size(lmm.data[1].X, 2)
    q = size(lmm.data[1].Z, 2)
    # print header
    print(io, "ID,Y,")
    for j in 1:(p-1)
        print(io, "X" * string(j) * ",")
    end
    for j in 1:(q-1)
        print(io, "Z" * string(j) * (j < q-1 ? "," : "\n"))
    end
    # print data
    for i in eachindex(lmm.data)
        obs = lmm.data[i]
        for j in 1:length(obs.y)
            # id
            print(io, i, ",")
            # Y
            print(io, obs.y[j], ",")
            # X data
            for k in 2:p
                print(io, obs.X[j, k], ",")
            end
            # Z data
            for k in 2:q-1
                print(io, obs.Z[j, k], ",")
            end
            print(io, obs.Z[j, q], "\n")
        end
    end
end